This notebook is a tutorial based on the following manuscript: 

[INVERSE BOUNDARY VALUE AND OPTIMAL CONTROL PROBLEMS ON
GRAPHS: A NEURAL AND NUMERICAL SYNTHESIS](https://arxiv.org/pdf/2206.02911). 

Partial Differential Equations (PDEs) are among the most studied mathematical models, representing many systems in science and engineering. A PDE describes the time evolution of a dynamical system defined over a spatial region.
PDEs and machine learning (ML) can mutually enhance each other. ML can be used to learn solutions to PDEs or to infer PDEs in inverse problems, while PDEs can inspire the design of ML algorithms.

As an example of the first case, where ML aids in solving PDEs, consider physics-informed neural networks (PINNs), which have gained significant attention in recent years. The idea behind PINNs is to integrate physical knowledge in the form of PDEs into the learning algorithm. The neural network models the solution of the PDE, and with automatic differentiation, the network is tuned to fit the data while adhering to the physical laws. After training, a PINN can generate solutions to a PDE given the initial and boundary conditions, eliminating the need for numerical simulation techniques like the finite element method. Our work is similar to PINNs in that we aim to learn the differential equation that generated the observational data. However, our approach differs because we do not assume prior knowledge of the form of the PDE, making it applicable in more general settings.

<img src="../imgs/vonKarman.png" width=500>

A cylinder is obstructing a flow and creating vortices as a result. This type of fluid dynamics can be modelled as a PDE. <a href="https://en.wikipedia.org/wiki/K%C3%A1rm%C3%A1n_vortex_street">Image source</a>

An example of how PDEs can inspire novel ML algorithms is the diffusion equation, which models information propagation over a topological space. The diffusion equation is a well-known PDE that describes the diffusion of fields such as energy, mass concentration, or information over a medium. It has been shown that the dynamics of graph neural networks (GNNs) can be interpreted as the numerical integration of the diffusion equation over a discretized space. GNNs are neural networks that compute the updated state of nodes in a graph based on their neighboring nodes, with message passing implemented over the edges. The underlying graph of a GNN is analogous to the discretization schemes used in numerical solutions of PDEs. This natural connection between GNNs and PDEs makes GNNs strong candidates for modeling the updates of dynamical systems described by unknown PDEs.

Dynamical systems are ubiquitous in nature and industral applications alike. Here are a short list of possible applications:

- Optimal resource allocation in networks
- Traffic flow and congestion models
- Brain computer interfaces
- Crowd dynamics in urban planning
- Heat transfer in industrial processes
- Water resource management
- Pollution dispersion
- Customer behavior modelling in markeing
- Pricing strategies
- Mean field game theory
- Option pricing

Our explicit goal is: using graph neural networks to learn a dyanmical system on a graph with boundary. This is done through a model called _Boundary Informed Graph Neural ODE_, or for short, **BigNode**.

Our method learns to represent the unknown PDE that generated the observations over space and time, treating it as a system of ODEs at each point in space. We model the observational data as the numerical integration of a GNN over an underlying graph, which is a discretization of the spatial domain. The main innovation of our approach lies in handling the known boundary conditions of the system. Unlike the standard PINN approach, which imposes boundary conditions as a penalty term in the loss function, our method addresses them in a structured way, similar to how boundary conditions are handled in numerical PDE solutions. We use the underlying graph with boundary nodes and directly enforce the boundary conditions through boundary-injected message passing layers of the GNN. Finally, the integration and backpropagation of the GNN over time are performed using the framework of Neural ODEs.


In [ ]:
import os
import random
import pickle

import numpy as np
import torch

from bignode_no_ctrl import BIGNODE
from utils_sysid import train, inference, compute_RMSE
from utils_visualization import visualize_state_sysid
from utils import experiment_init
from IPython.display import Image, display
import matplotlib.pyplot as plt

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = "data/linear"

# Load graph data
with open(os.path.join(ROOT_DIR, DATA_DIR, "graph_obj.pickle"), "rb") as handle:
    graph_obj = pickle.load(handle)
num_nodes_int = graph_obj["num_nodes_int"]
num_nodes_bound = graph_obj["num_nodes_bound"]

In [ ]:
# Graphs are represented in the following format which is friendly to Torch. 
graph_obj

<!-- ![](../imgs/bignode_img1.png) -->
<img src="../imgs/bignode_img1.png" width=500>
<!-- from IPython.display import Image, display
display(Image(filename="/Users/boofebeena/Pictures/Plots/bignode_img1.png")) -->

[NEED CLEAR PROBLEM STATEMENT AS TO WHAT'S UP WITH BOUNDARY NODES]

In [ ]:
# Load system data
with open(os.path.join(ROOT_DIR, DATA_DIR, "system_data.pickle"), "rb") as handle:
    system_data = pickle.load(handle)

print(system_data.keys())
print(system_data['system_name']) # We're working with synthetic head diffusion data
print(system_data['timestamps'].shape) # 1001 timestamps
print(system_data['U']) # U denotes control nodes, empty in this example
print(system_data['X_int'].shape) # Observations on graph interior with 10 gray nodes
print(system_data['X_bound'].shape) # Two boundary nodes in cyan

Our data comes from a numerical simulation of the standard diffusion formula:

$\dot{\bf{x}} = - \kappa \Delta \bf{x}$

where $\kappa$ is a diffusitivity coefficient, and $\Delta = \mathrm{grad}^*\mathrm{grad} = - \mathrm{div}\, \mathrm{grad}$ is the standard Laplacian. 

In [ ]:
SYSTEM_TYPE = "linear"
MESSAGE_DIM = 32
EMBED_DIM = 32
REGULARIZATION = True
EXPERIMENT_NAME = "bignode_reg_linear" if REGULARIZATION else "bignode_linear"
EPOCHS = 2000

random.seed(23)
np.random.seed(23)
torch.manual_seed(23)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(23)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Initializing the model
bignode = BIGNODE(input_dim=1, message_dim=MESSAGE_DIM, embed_dim=EMBED_DIM, bvp_type="dirichlet",
                    num_nodes_int=num_nodes_int, num_nodes_bound=num_nodes_bound, device=device)

# Initializing the experiment
experiment_dirs = experiment_init(experiment_name=EXPERIMENT_NAME, graph_obj=graph_obj, system_data=system_data)

[NEED A VISUAL FOR MODEL ARCHITECTURE]

[BOUNDARY INJECTION]

In [ ]:
# Training
model, losses = train(graph_obj=graph_obj,
                system_data=system_data,
                regularization=REGULARIZATION,
                model=bignode,
                model_dir=experiment_dirs["model_dir"],
                epochs=EPOCHS)

In [ ]:
plt.title('Train loss convergence')
plt.plot(range(len(losses)), losses)

In [ ]:
experiment_dirs["visualization_dir"]

In [ ]:
# Training evaluation
X_int_hat = inference(graph_obj, system_data, model)
visualize_state_sysid(system_type=SYSTEM_TYPE,
                        graph_obj=graph_obj,
                        X_bound=system_data["X_bound"],
                        X_int=system_data["X_int"],
                        X_int_hat=X_int_hat,
                        visualization_dir=experiment_dirs["visualization_dir"])

[NEED MODEL CONVERGENCE PLOT]

In [ ]:
art_plots_dir = os.path.join(ROOT_DIR, 'artifacts/bignode_reg_linear_2024-11-09-16-06-24/visualization/')
node_plots = os.listdir(art_plots_dir)

for filename in node_plots:
    display(Image(filename=os.path.join(art_plots_dir, filename), width=450))

[NONLINEAR CASE]

In [ ]:
SYSTEM_TYPE = "nonlinear"
DATA_DIR = "data/nonlinear"

# Load graph data
with open(os.path.join(ROOT_DIR, DATA_DIR, "graph_obj.pickle"), "rb") as handle:
    graph_obj = pickle.load(handle)
num_nodes_int = graph_obj["num_nodes_int"]
num_nodes_bound = graph_obj["num_nodes_bound"]

# Load system data
with open(os.path.join(ROOT_DIR, DATA_DIR, "system_data.pickle"), "rb") as handle:
    system_data = pickle.load(handle)

In [ ]:
MESSAGE_DIM = 64
EMBED_DIM = 64
REGULARIZATION = True
EXPERIMENT_NAME = "bignode_reg_nonlinear" if REGULARIZATION else "bignode_nonlinear"

# Initializing the model
bignode = BIGNODE(input_dim=1, message_dim=MESSAGE_DIM, embed_dim=EMBED_DIM, bvp_type="dirichlet",
                    num_nodes_int=num_nodes_int, num_nodes_bound=num_nodes_bound, device=device)

# Initializing the experiment
experiment_dirs = experiment_init(experiment_name=EXPERIMENT_NAME, graph_obj=graph_obj, system_data=system_data)

In [ ]:
# Training
model, losses = train(graph_obj=graph_obj,
                system_data=system_data,
                regularization=REGULARIZATION,
                model=bignode,
                model_dir=experiment_dirs["model_dir"],
                epochs=EPOCHS)

In [ ]:
plt.title('Train loss convergence')
plt.plot(range(len(losses)), losses)

In [ ]:
ROOT_DIR

In [ ]:
art_plots_dir = os.path.join(ROOT_DIR, 'artifacts/bignode_reg_nonlinear_2024-11-10-16-31-59/visualization/')
node_plots = os.listdir(art_plots_dir)

for filename in node_plots:
    display(Image(filename=os.path.join(art_plots_dir, filename), width=450))

[REGULARIZATION]

 [NEED CONCLUSION AN COMMENTARY]

<img src="../imgs/lin.png" width=800>


<img src="../imgs/lin.png" width=800>
